In [2]:
#import modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn.model_selection as ms
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn import ensemble
import xgboost
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [3]:
#Import dataset
wolabor2016 = pd.read_csv('CDC2016wolabor.csv')

In [4]:
#get rid of 'U' of AB_NICU
wolabor2016 =  wolabor2016[wolabor2016['AB_NICU'].isin(['Y', 'N'])]

In [7]:
# function to split out holdout test set
def split_sets(dataframe, seed, test_prop=0.1): 
    '''
    - A function that splits specifically a dataframe into a train and test portion
    - Requires multiple assignment: train, test
    ---------------
    - dataframe: dataframe to be split
    - seed: set seed for reproducability
    - test_prop: takes a float - proportion of dataframe that should be allocated to the test set
    '''

    np.random.seed(seed)
    testIdxes = np.random.choice(range(0,dataframe.shape[0]), size=round(dataframe.shape[0]*test_prop), replace=False)
    trainIdxes = list(set(range(0,dataframe.shape[0])) - set(testIdxes))

    train = dataframe.iloc[trainIdxes,:]
    test  = dataframe.iloc[testIdxes,:]
    
    return train, test

In [8]:
train, test = split_sets(wolabor2016, 0, test_prop=0.1)

In [ ]:
print(train.shape)
print(test.shape)

In [9]:
dsample = train.copy()

In [10]:
#Downsample function, thanks Bettina and Aron!
def downsample_df (df):

    '''
    Remove undefined information on NICU admissions (AB_NICU == 'U'),
    create a binary target vector, and create a "balanced" dataframe
    with all NICU admissions and matching numbers of randomly selected non-NICU admissions.
    '''

    import pandas as pd
    import numpy as np

    # remove unknown class from df
    df_no_unknown = df[df['AB_NICU'].isin(['Y', 'N'])]

    # Create binary target vector, NICU = yes classified as class 0
    df_y_n = np.where((df_no_unknown['AB_NICU'] == 'Y'), 0, 1)

    # Get indicies of each class' observations
    index_class0 = np.where(df_y_n == 0)[0]
    index_class1 = np.where(df_y_n == 1)[0]

    # Get numbers of observations in class 0
    n_class0 = len(index_class0)

    # Randomly sample the same number of observations from class 1 as in class 0, without replacement
    np.random.seed(0)
    index_class1_downsampled = np.random.choice(index_class1, size=n_class0, replace=False)

    # Create dataframes for NICU and downsampled non-NICU
    df_defect = df_no_unknown.iloc[index_class0]
    df_adj_NONdefect = df_no_unknown.iloc[index_class1_downsampled]

    # Append into 1 dataframe
    df_downsampled = df_defect.append(df_adj_NONdefect)

    return df_downsampled

In [11]:
#Downsampled
dsample = downsample_df(dsample)

In [12]:
#check
print(sum(train.AB_NICU == 'Y'))  
print(dsample.shape)   
print(sum(dsample.AB_NICU == 'Y'))   

310649
(621298, 54)
310649


In [13]:
#LabelEncoding Function. Thanks Ira!
def LabelEncoding(dataframe):
    '''
    Function that takes a dataframe and transforms it with label encoding on all the categorical features.
    '''
    
    #create a list using object types since dataframe.dtypes.value_counts() only shows objects and int64
    objlist = list(dataframe.select_dtypes(include=['object']).columns)
    
    #change type then transform column using cat codes
    for col in objlist:
        dataframe[col] = dataframe[col].astype('category')
        dataframe[col] = dataframe[col].cat.codes
    
    return dataframe

In [14]:
#Label Encoded
dsample = LabelEncoding(dsample)
dtest = LabelEncoding(test)

/Users/bee/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/Users/bee/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [ ]:
'''
def targetchoice(column,dataframe):
    '''
    Takes a column and a dataframe, returns four values for;
    x_train, X_test, y_train, and y_test
    '''
    #Cutting the data and target dataframes
    sample_data = dataframe.loc[:, dsample.columns != column ]
    sample_target = dataframe.loc[:,column]
    
    #assigning to variables
    X_train, X_test, y_train, y_test = train_test_split(sample_data, sample_target, test_size=0.2, random_state=0)
    
    #appending to a list to return for multi-assignment
    varlist = []
    varlist.append(X_train)
    varlist.append(X_test)
    varlist.append(y_train)
    varlist.append(y_test)
    return varlist

#format is "X_train, X_test, y_train, y_test = targetchoice()"
'''

In [ ]:
#X_train, X_test, y_train, y_test = targetchoice('AB_NICU',dsample)

In [15]:
X_train = dsample.drop('AB_NICU', axis=1)
y_train = dsample['AB_NICU']
X_test = dtest.drop('AB_NICU', axis=1)
y_test = dtest['AB_NICU']

In [16]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(621298, 53)
(395168, 53)
(621298,)
(395168,)


In [17]:
#XGBoost initial fit 
xgb = XGBClassifier()
xgb.set_params(random_state=0)
xgb.fit(X_train, y_train)
print("The training error is: %.5f" % (1 - xgb.score(X_train, y_train)))
print("The test error is: %.5f" % (1 - xgb.score(X_test, y_test)))

The training error is: 0.32243
The test error is: 0.24794


In [18]:
# set the parameter grid
xgb_param_grid ={'learning_rate': [0.05],
                 'max_depth': [5],
                 'min_child_weight': [4],
                 'n_estimators': [300]}

#grid search
grid_search_xgb = GridSearchCV(xgb, xgb_param_grid, scoring='accuracy', cv= 5, n_jobs=-1, return_train_score = True)
%time grid_search_xgb.fit(X_train, y_train)

KeyboardInterrupt: 

In [ ]:
# get the best parameters
print(grid_search_xgb.best_params_)
print(grid_search_xgb.best_score_)

In [ ]:
# get the training/test errors
print("The training error is: %.5f" % (1 - grid_search_forest.best_estimator_.score(X_train, y_train)))
print("The test error is: %.5f" % (1 - grid_search_forest.best_estimator_.score(X_test, y_test)))

In [ ]:
#Prediction with tuned hyperparameters
grid_xgb_pred = grid_xgb.predict(X_test)

In [ ]:
printErrors(Y_test, grid_xgb_pred)

In [ ]:
# Get numerical feature importances
importances_xgb = list(xgb.feature_importances_)

# List of tuples with variable and importance
feature_importances_xgb = [(feature, round(importance, 5)) for feature, importance in zip(X_train.columns, importances_xgb)]

# Sort the feature importances by most important first
xgb_feature_importances = sorted(feature_importances_xgb, key = lambda x: x[1], reverse = True )

# Print out the feature and importances 
[print('Variable: {:10} Importance: {}'.format(*pair)) for pair in xgb_feature_importances]

In [ ]:
xgb_feature_importances_top20 = xgb_feature_importances[:20]
featureNames, featureScores = zip(*list(xgb_feature_importances_top20))
xgb_feature_importances_top20

In [ ]:
plt.barh(range(len(featureScores)), featureScores, tick_label=featureNames)
plt.gca().invert_yaxis()
plt.title('feature importance')
plt.ylabel('Features')
plt.xlabel('Importance Score')
plt.title('Feature Importances')
plt.savefig('xgbFI.png')

In [ ]:
xgb_params_tuned_model = grid_search_xgb.best_estimator_
xgb_feature_importance = 100.0 * (xgb_params_tuned_model.feature_importances_ / xgb_params_tuned_model.feature_importances_.max())
xgb_important_features = X_train.columns[xgb_feature_importance >= 2]
xgb_unimportant_features = X_train.columns[xgb_feature_importance < 2]

In [ ]:
xgb_important_features

In [ ]:
#confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, gs_logit.best_estimator_.predict(X_test))
cm